In [7]:
import import_ipynb
import copy
from builtins import range

import corner as Co
import edge as Ed
import facelet as Fc
import facecube as FC

importing Jupyter notebook from corner.ipynb
importing Jupyter notebook from facelet.ipynb
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53)
importing Jupyter notebook from edge.ipynb
importing Jupyter notebook from facecube.ipynb
importing Jupyter notebook from color.ipynb


In [46]:
class CubieCube(object):
    #cubie level
    def __init__(self, cp=None, co=None, ep=None, eo=None):
        # corner permutation
        self.cp = copy.copy(cp) if cp else ['URF', 'UFL', 'ULB', 'UBR', 'DFR', 'DLF', 'DBL', 'DRB']

        # corner orientation
        self.co = copy.copy(co) if co else [0, 0, 0, 0, 0, 0, 0, 0]

        # edge permutation
        self.ep = copy.copy(ep) if ep else ['UR', 'UF', 'UL', 'UB', 'DR', 'DF', 'DL', 'DB', 'FR', 'FL', 'BL', 'BR']

        # edge orientation
        self.eo = copy.copy(eo) if eo else [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]        
    
    def to_facelet(self):
        s = ['0' for i in range(54)]
        # set the centers
        s[Fc.U5] = 'U'
        s[Fc.R5] = 'R'
        s[Fc.F5] = 'F'
        s[Fc.D5] = 'D'
        s[Fc.L5] = 'L'
        s[Fc.B5] = 'B'
        
        # assign the value by each cubie
        # corner
        for c in Co.corner_keys:
            Cubie_at_c = self.cp[Co.corners[c]]
            color_at_face = ''
            for i in range(3):
                face = Co.corner_facelets[c][i]
                index = i
                if self.co[Co.corners[c]] % 3 == 1:  # clockwise
                    index = (i - 1 + 3)%3
                if self.co[Co.corners[c]] % 3 == 2:  # counterclockwise
                    index = (i - 2 + 3)%3   
                num_of_color_at_face = Co.corner_facelets[Cubie_at_c][index]
                for i in Fc.facelets.keys():
                    if Fc.facelets[i] == num_of_color_at_face:
                        color_at_face = i
                s[face] = color_at_face        
        # edge
        for e in Ed.edge_keys:
            Cubie_at_e = self.ep[Ed.edges[e]]
            color_at_face = ''
            for i in range(2):
                face = Ed.edge_facelets[e][i]
                index = i
                if self.eo[Ed.edges[e]] % 2 == 1:  # flipped
                    index = (i - 1 + 2) % 2
                num_of_color_at_face = Ed.edge_facelets[Cubie_at_e][index]
                for i in Fc.facelets.keys():
                    if Fc.facelets[i] == num_of_color_at_face:
                        color_at_face = i
                s[face] = color_at_face        

        seq = "".join(s)   
        Q = FC.FaceCube(seq)
        return Q         

In [49]:
cpU = ['UBR', 'URF', 'UFL', 'ULB', 'DFR', 'DLF', 'DBL', 'DRB']
coU = [0, 0, 0, 0, 0, 0, 0, 0]
epU = ['UB', 'UR', 'UF', 'UL', 'DR', 'DF', 'DL', 'DB', 'FR', 'FL', 'BL', 'BR']
eoU = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
a = CubieCube(cpU, coU, epU,eoU)
q = a.to_facelet()
print(q.to_2dstring())

   UUU
   UUU
   UUU
FFFRRRBBBLLL
LLLFFFRRRBBB
LLLFFFRRRBBB
   DDD
   DDD
   DDD



In [27]:
cpU = [corner.UBR, corner.URF, corner.UFL, corner.ULB, corner.DFR, corner.DLF, corner.DBL, corner.DRB]
coU = [0, 0, 0, 0, 0, 0, 0, 0]
epU = [edge.UB, edge.UR, edge.UF, edge.UL, edge.DR, edge.DF, edge.DL, edge.DB, edge.FR, edge.FL, edge.BL, edge.BR]
eoU = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

cpR = [corner.DFR, corner.UFL, corner.ULB, corner.URF, corner.DRB, corner.DLF, corner.DBL, corner.UBR]
coR = [2, 0, 0, 1, 1, 0, 0, 2]
epR = [edge.FR, edge.UF, edge.UL, edge.UB, edge.BR, edge.DF, edge.DL, edge.DB, edge.DR, edge.FL, edge.BL, edge.UR]
eoR = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

cpF = [corner.UFL, corner.DLF, corner.ULB, corner.UBR, corner.URF, corner.DFR, corner.DBL, corner.DRB]
coF = [1, 2, 0, 0, 2, 1, 0, 0]
epF = [edge.UR, edge.FL, edge.UL, edge.UB, edge.DR, edge.FR, edge.DL, edge.DB, edge.UF, edge.DF, edge.BL, edge.BR]
eoF = [0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]

cpD = [corner.URF, corner.UFL, corner.ULB, corner.UBR, corner.DLF, corner.DBL, corner.DRB, corner.DFR]
coD = [0, 0, 0, 0, 0, 0, 0, 0]
epD = [edge.UR, edge.UF, edge.UL, edge.UB, edge.DF, edge.DL, edge.DB, edge.DR, edge.FR, edge.FL, edge.BL, edge.BR]
eoD = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

cpL = [corner.URF, corner.ULB, corner.DBL, corner.UBR, corner.DFR, corner.UFL, corner.DLF, corner.DRB]
coL = [0, 1, 2, 0, 0, 2, 1, 0]
epL = [edge.UR, edge.UF, edge.BL, edge.UB, edge.DR, edge.DF, edge.FL, edge.DB, edge.FR, edge.UL, edge.DL, edge.BR]
eoL = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

cpB = [corner.URF, corner.UFL, corner.UBR, corner.DRB, corner.DFR, corner.DLF, corner.ULB, corner.DBL]
coB = [0, 0, 1, 2, 0, 0, 2, 1]
epB = [edge.UR, edge.UF, edge.UL, edge.BR, edge.DR, edge.DF, edge.DL, edge.BL, edge.FR, edge.FL, edge.UB, edge.DB]
eoB = [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1]
